##1. Mounting Data

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


##2. Importing Libraries

In [ ]:
import numpy as np
import pandas as pd

##3. Defining Path and Raw data

In [ ]:
path = '/gdrive/My Drive/ML1 Course1 Sept 2020/Assignments/Data/'

In [ ]:
raw_data = pd.read_csv(path + 'Iris.csv')
raw_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


##4. Dropping Unwanted Data

In [ ]:
raw_data = raw_data.drop(columns=["Id"])

##5. Shuffling the Data
Shuffling data serves the purpose of reducing variance and making sure that models remain general and overfit less. 
By shuffling your data, you ensure that each data point creates an "independent" change on the model, without being biased by the same points before them.

In [ ]:
from sklearn.utils import shuffle
raw_data = shuffle(raw_data)

raw_data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
136,6.3,3.4,5.6,2.4,Iris-virginica
86,6.7,3.1,4.7,1.5,Iris-versicolor
11,4.8,3.4,1.6,0.2,Iris-setosa
44,5.1,3.8,1.9,0.4,Iris-setosa
119,6.0,2.2,5.0,1.5,Iris-virginica


##6. Encoding Categorical Data

In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
raw_data["Species"] = labelencoder.fit_transform(raw_data["Species"])
raw_data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
136,6.3,3.4,5.6,2.4,2
86,6.7,3.1,4.7,1.5,1
11,4.8,3.4,1.6,0.2,0
44,5.1,3.8,1.9,0.4,0
119,6.0,2.2,5.0,1.5,2


##7. Seperating X and Y

In [ ]:
y = raw_data.iloc[:, raw_data.shape[1]-1:raw_data.shape[1]]
x = raw_data.iloc[:, 1:raw_data.shape[1]]
print(x.shape, y.shape)

(150, 4) (150, 1)


In [ ]:
x.head()

,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
136,3.4,5.6,2.4,2
86,3.1,4.7,1.5,1
11,3.4,1.6,0.2,0
44,3.8,1.9,0.4,0
119,2.2,5.0,1.5,2


In [ ]:
y.head()

,Species
136,2
86,1
11,0
44,0
119,2


##8. Creating Classes

In [ ]:
classes = np.unique(y)
classes = classes.size
print(classes)

3


##9. Splitting Test and Train Data

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=2)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(112, 4) (38, 4) (112, 1) (38, 1)


##10. Changing the Type of X and Y

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

##11. Inserting Bias Term

In [ ]:
rows = x_train.shape[0]
x_train = np.insert(x_train, 0, values=np.ones(rows), axis=1)
print(x_train.shape)

(112, 5)


##12. Defining ComputeCost, Sigmoid and Gradient

In [ ]:
theta = np.zeros((classes, x_train.shape[1]))
print(theta.shape)

(3, 5)


In [ ]:
#Model
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
#Error
def ComputeCost(theta, x, y):
    theta = np.matrix(theta)
    x = np.matrix(x)
    y = np.matrix(y)
    # (-y*log(g(z))) --> when expected output (y)=1
    first = np.multiply(-y, np.log(sigmoid(x * theta.T))) 
    # ((1-y)*(1-log(g(z)))) --> when expected output (y)=0
    second = np.multiply((1 - y), np.log(1 - sigmoid(x * theta.T)))
    cost = np.sum(first - second)
    cost = cost/(2*len(x)) 
    return cost

In [ ]:
print(ComputeCost(theta, x_train, y_train))

1.0397207708399179


In [ ]:
def gradient(theta, x, y):
    
    theta = np.matrix(theta)
    x = np.matrix(x)
    y = np.matrix(y)
    learning_rate = 1
        
    error = sigmoid(x * theta.T) - y
    grad = learning_rate*((x.T * error) / len(x))
    
    return grad

##13. Defining Minimize method

In [ ]:
from scipy.optimize import minimize
def one_vs_all(x, y, classes, new_theta):
    params = x.shape[1]
    rows = x.shape[0]
      
    for i in range(0, classes):
        curr_theta = new_theta[i,:]
        
        y_i = np.array([1 if label == i else 0 for label in y]) 
        y_i = np.reshape(y_i, (rows, 1))
                
        # minimize the objective function  --> Gradient descent
        fmin = minimize(fun=ComputeCost, x0=curr_theta, args=(x, y_i), method='TNC', jac=gradient)
        theta[i,:] = fmin.x
        
        print(fmin.fun)
    
    return theta 

In [ ]:
theta = one_vs_all(x_train, y_train, classes, theta)

2.100443796737246e-08
0.09592642402024668
7.328080902615946e-09


In [ ]:
print('new theta = {}'.format(theta))

new theta = [[  3.45208845  10.35675708  -9.60340794  -4.5595971   -6.37278514]
 [ 16.85530852  -8.39549868   3.78080299   7.47037439 -16.58489639]
 [-32.79431648 -11.50538653  -3.77498101  12.16908571  41.35025238]]


##14. Predicting

In [ ]:
def predict_all(x, theta):

    x = np.matrix(x)
    theta = np.matrix(theta)
    
    h = sigmoid(x * theta.T)
    
    h_argmax = np.argmax(h, axis=1)
    
    return h_argmax, h 

In [ ]:
y_prediction, h = predict_all(x_train, theta)
print(y_prediction, h)

[[1]
 [1]
 [2]
 [1]
 [0]
 [2]
 [0]
 [1]
 [0]
 [1]
 [2]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [2]
 [1]
 [0]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [2]
 [2]
 [0]
 [2]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [2]
 [2]
 [1]
 [2]
 [2]
 [2]
 [1]
 [1]
 [2]
 [2]
 [0]
 [2]
 [2]
 [1]
 [2]
 [1]
 [1]
 [2]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]
 [2]
 [0]
 [1]
 [1]
 [1]
 [1]
 [2]
 [1]
 [0]
 [1]
 [2]
 [2]
 [2]
 [0]
 [0]
 [1]
 [0]
 [1]
 [2]
 [0]
 [1]
 [1]
 [2]
 [2]
 [1]
 [2]
 [2]
 [0]
 [2]
 [2]
 [2]
 [1]
 [0]
 [1]
 [2]
 [1]
 [0]
 [1]
 [0]
 [0]] [[1.22417483e-08 6.66395919e-01 6.38754806e-13]
 [1.30212109e-08 8.29283916e-01 8.76279672e-10]
 [3.42655751e-18 6.84410413e-02 1.00000000e+00]
 [1.85445658e-11 9.81582458e-01 1.31848107e-12]
 [1.00000000e+00 1.07280480e-02 2.32570449e-33]
 [1.18897770e-17 4.04377702e-03 1.00000000e+00]
 [1.00000000e+00 2.26427601e-05 7.25028417e-36]
 [9.62002803e-11 9.70347734e-01 1.65150418e-11]
 [9.99999995e-01 3.34637340e-01 1.32992291e-31]
 [3.710

##15. Calculating Accuracy

In [ ]:
def accuracy(y_prediction, y):
  correct = [1 if a == b else 0 for (a, b) in zip(y_prediction, y)]
  acc = (sum(correct)) / (len(correct))
  return acc

In [ ]:
accuracy = acc(y_prediction, y_train)
print ("Training data accuracy={} %".format(accuracy * 100))

Training data accuracy=100.0 %


In [ ]:
t_x = np.array(x_test)
t_y = np.array(y_test)

t_rows = t_x.shape[0] #Finding the number of Rows (data samples)
t_x = np.insert(t_x, 0, values=np.ones(t_rows), axis=1) #inserting ones into x
t_y_pred, h= predict_all(t_x, theta)
print ("Test data accuracy={} %".format(acc(t_y_pred, t_y) * 100))

Test data accuracy=100.0 %
